In [11]:
# -*- coding: utf-8 -*-
"""
Reformats ANAT network output

python 2get_subnets.py terminal1 terminaln ... anchor ANATfilenameWithoutextension

creates a .path pkl file of a dictionary of paths per terminal
"""

import pickle
import collections
import itertools
import lxml.etree as etree
import os
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from random import seed, sample
from datetime import date
import sys



SPECIES = "S_cerevisiae"
# SPECIES='H_sapiens'
DATE=str(date.today().strftime("%d/%m/%Y"))
HOME_DIR='G:'+os.sep+'Il Mio Drive'+os.sep+'SECRET-ITN'+os.sep+'Projects'+os.sep+\
    'network_signing'+os.sep+'Validations'+os.sep+'ANAT_validation_pipeline'+os.sep
#HOME_DIR='G:'+os.sep+'My Drive'+os.sep+'SECRET-ITN'+os.sep+'Projects'+os.sep+\
 #   'network_signing'+os.sep+'SIGNAL Validations target sign reconstruction'\
  #      +os.sep+'ANAT_validation_pipeline'+os.sep
MAIN_DATA_DIR = 'G:'+os.sep+'Il Mio Drive'+os.sep+'SECRET-ITN'+os.sep+'Projects'+os.sep+'Data'+os.sep+SPECIES+os.sep
#MAIN_DATA_DIR = 'G:'+os.sep+'My Drive'+os.sep+'SECRET-ITN'+os.sep+'Projects'+os.sep+'Data'+os.sep+SPECIES+os.sep
#INPUT_DIR =  HOME_DIR+ '1 Build Pacitaxel target - fosfoprotein paths'
os.chdir(HOME_DIR)

with open( MAIN_DATA_DIR+'alias_2geneid.pkl', 'rb') as f:
    alias_2geneid = pickle.load(f)

# network=pd.read_csv(MAIN_DATA_DIR +SPECIES+'.net',sep="\t", header=None)
# graph = nx.from_pandas_edgelist(network.reset_index(), 0, 1, 2)
f=open(HOME_DIR+'1 ANAT Build networks'+os.sep+'Input'+os.sep+'TLManchors.txt')
lines=[x.strip() for x in f.readlines()]
ANCHORS=lines
f.close()
TLMphenotypesdf=pd.read_csv(HOME_DIR+'1 ANAT Build networks'+os.sep+'Input'+os.sep+'TLMphenotypesnodupes.txt', header=0, sep='\t')
TERMS=list(TLMphenotypesdf[TLMphenotypesdf.columns[0]].values)

    # For TUBB Paclitaxtel validaiton
#    FILENAME = 'TUBB_v_all'
 #   ANCHORS='TUBB'
  #  TERMS=['SMAD3','JUN','TP53','RPS6KB1','EGFR','MARCKS','PRAS40','PTP2C','IKBA',
   #         'GSK3A','AKT1','HSPB1','H2A.X','MAPK11','MEK1','STAT3','ERK1','RSK1','CREB1','FAK1']


In [18]:
#%%  with networkx (has revealed the bug with TLM data)
FILENAME='TLM_strong_normal'#'TLM_all_KOterms'#
network_file='1 ANAT Build networks'+os.sep+'ANAT output'+os.sep+FILENAME


# Read the CSV edges file 
columns = ['ID1', 'ID2']
edges_df = pd.read_csv(network_file+'.csv', usecols=columns)
graph_symbols=nx.from_pandas_edgelist(edges_df, source='ID1', target='ID2')

# Read the CSV nodes file and select the desired columns and rows
nodesfile=HOME_DIR+'1 ANAT Build networks'+os.sep+'ANAT output'+os.sep+FILENAME+'_nodes.csv'
columns = ['name', 'status','xrefID','xrefName']
nodes_df = pd.read_csv(nodesfile, usecols=columns)

#%% v2 debugging: checking difference between ANAT output's terms and sources and original terms and sources
with open( MAIN_DATA_DIR+'symbol_2geneid.pkl', 'rb') as f:
    symbol_2geneid = pickle.load(f)
id_2alias={y:x for (x,y) in symbol_2geneid.items()}

In [19]:
#%% check if some nodes in ANAT output graph are not present in ancohors or terminals
# to do this, first check that all nodes in the nodes table from cytoscape
# are in the edges table as well,

print('checking cytoscape nodes against  cytoscape edges using symbols')
for genename in nodes_df.xrefName:
    if not genename in graph_symbols.nodes:
        print('qlcs non va!! Geni in NODES table not present in EDGES table:', genename)
for genename in graph_symbols.nodes:
    if not genename in list(nodes_df.xrefName):
        print('qlcs non va! geni in edges non presenti in nodes', genename)

# usando questo mi rendo conto che l unico problema e' il nodo 'TEN1' o '850696'. per TLM allkoterms
#menter per TLM strong_normal è il nodo EST2 (e infatti vedo dal network che non è riuscito a connetterlo, quindi lo elimino)

checking cytoscape nodes against  cytoscape edges using symbols
qlcs non va!! Geni in NODES table not present in EDGES table: EST2


In [17]:
nodes_df[nodes_df['xrefName']=='TEN1']

,name,status,xrefID,xrefName
28,TEN1,ANCHOR,850696,TEN1


In [38]:
'TEN1' in [id_to_alias[int(a)] for a in ANCHORS]

True

In [20]:
nodes_df[nodes_df['xrefName']=='EST2']

,name,status,xrefID,xrefName
97,EST2,ANCHOR,851028,EST2


In [30]:
#%% controlla che ci sia un rapporto 1:1 tra i nodi du graph_symbols e di nodes_df
# ORA sappiamo che c'è discrepancy qunado un anchor rimane scollegato.
print('check for 1:1 mapping between nodes between node file and edge file from anat cytoscape output')
print('# nodes from nodes file:',nodes_df.shape,'# nodes from edges file:', len(graph_symbols.nodes) ) # discrepanza di 1, a'accettabile (?)

for genename in nodes_df.xrefName:
    if not genename in graph_symbols.nodes:
        print('qlcs non va!!', genename, 'in Nodes file but not in Edges file')
# controllo ridondante, TEN1 (850696) viene PERSO dall'edges file. forse xke semplicemente NON e' connesso!


check for 1:1 mapping between nodes between node file and edge file from anat cytoscape output
# nodes from nodes file: (464, 4) # nodes from edges file: 463
qlcs non va!! TEN1 in Nodes file but not in Edges file


In [16]:
#%% vediamo ora se tutti i nodi di TERMS almeno sono dentro nodes:
print(len(TERMS),len(nodes_df[nodes_df.status=='TERMINAL']))
for genename in list(TLMphenotypesdf[TLMphenotypesdf.columns[0]].values): # per prendere i genenames non tradotti
    if not genename in list(nodes_df.xrefName):
        print('qlcs non va! geni in original terms non presenti in nodes', genename)
        # mancano cmq molti geni dei temrinals! aggiungere manualmente a ANAT e rilanciare

461 419
qlcs non va! geni in terms non presenti in nodes FMP26
qlcs non va! geni in terms non presenti in nodes IPK2
qlcs non va! geni in terms non presenti in nodes KEM1
qlcs non va! geni in terms non presenti in nodes MDN1
qlcs non va! geni in terms non presenti in nodes PHO88
qlcs non va! geni in terms non presenti in nodes SUR4
qlcs non va! geni in terms non presenti in nodes TFP1
qlcs non va! geni in terms non presenti in nodes TID3
qlcs non va! geni in terms non presenti in nodes YBR284W
qlcs non va! geni in terms non presenti in nodes YBR292C
qlcs non va! geni in terms non presenti in nodes YDL119C
qlcs non va! geni in terms non presenti in nodes YDR115W
qlcs non va! geni in terms non presenti in nodes YDR532C
qlcs non va! geni in terms non presenti in nodes YEL057C
qlcs non va! geni in terms non presenti in nodes YGL039W
qlcs non va! geni in terms non presenti in nodes YGR042W
qlcs non va! geni in terms non presenti in nodes YHL012W
qlcs non va! geni in terms non presenti in no

In [17]:
#%% vediamo ora se tutti i nodi di TERMS almeno sono dentro il graph (alcune di ste cells potrebbero ripeters):
print(len(TERMS),len(nodes_df[nodes_df.status=='TERMINAL']))
for genename in list(TLMphenotypesdf[TLMphenotypesdf.columns[0]].values): # per prendere i genename s non tradotti
    if not genename in graph_symbols.nodes:
        print('qlcs non va! geni in terms non presenti in nodes', genename)
        # mancano cmq molti geni dei temrinals! aggiungere manualmente a ANAT e rilanciare

# AOCCHIO I GENES DELLE DUE CELLS SOPRA OSNO GLI STESSI
#%%

461 419
qlcs non va! geni in terms non presenti in nodes FMP26
qlcs non va! geni in terms non presenti in nodes IPK2
qlcs non va! geni in terms non presenti in nodes KEM1
qlcs non va! geni in terms non presenti in nodes MDN1
qlcs non va! geni in terms non presenti in nodes PHO88
qlcs non va! geni in terms non presenti in nodes SUR4
qlcs non va! geni in terms non presenti in nodes TEN1
qlcs non va! geni in terms non presenti in nodes TFP1
qlcs non va! geni in terms non presenti in nodes TID3
qlcs non va! geni in terms non presenti in nodes YBR284W
qlcs non va! geni in terms non presenti in nodes YBR292C
qlcs non va! geni in terms non presenti in nodes YDL119C
qlcs non va! geni in terms non presenti in nodes YDR115W
qlcs non va! geni in terms non presenti in nodes YDR532C
qlcs non va! geni in terms non presenti in nodes YEL057C
qlcs non va! geni in terms non presenti in nodes YGL039W
qlcs non va! geni in terms non presenti in nodes YGR042W
qlcs non va! geni in terms non presenti in nodes

In [18]:
paths_per_anchor ={}
for ANCHOR in ANCHORS:
    print('fidind path from:', ANCHOR)
    # Find all paths from ANCHOR to each final node
    
    # save all shortest paths (one more layer of depth to dictionary)
    paths_per_anchor[ANCHOR] = {node: nx.shortest_path(graph, ANCHOR, node) for node in TERMS}
    print(paths_per_anchor[ANCHOR])
    break
        
#%%

fidind path from: 850934


NodeNotFound: Either source 850934 or target 852205 is not in G

In [1]:
ANCHOR='850696'
alias_paths_to = {id_to_alias[int(key)]:[id_to_alias[int(y)] for y in val[0]] for (key, val) in paths_per_anchor[ANCHOR].items()}

NameError: name 'paths_per_anchor' is not defined